# Analiza košarkarskih podatkov

Za projektno nalogo bom analiziral podatke, ki sem jih pridobil iz spletne strani [Basketball Reference](https://www.basketball-reference.com/), natančneje podatke iz tabele, ki prikazuje statistiko vsakega igralca v sezoni 2023/24, tako redni del sezone kot končnica.

Z uporabo različnih metod bom raziskal, kako se nekatere kategorije košarkarskih podatkov med seboj povezujejo in odražajo v igri.

In [2]:
#naložimo potrebne pakete za obdelavo podatkov
import pandas as pd
import matplotlib.pyplot as plt

# uvozimo datoteko, kjer imamo podatke igralcev
redni_del = pd.read_csv("redni_del_sezone.csv")
koncnica = pd.read_csv("koncnica.csv")

## Redni del sezone

Za začetek si poglejmo redni del sezone. Naša tabela zaenkrat izgleda tako:

In [3]:
redni_del

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
0,Joel Embiid,29.0,PHI,C,39.0,39.0,33.6,11.5,21.8,0.529,...,2.4,8.6,11.0,5.6,1.2,1.7,3.8,2.9,34.7,AS
1,Luka Dončić,24.0,DAL,PG,70.0,70.0,37.5,11.5,23.6,0.487,...,0.8,8.4,9.2,9.8,1.4,0.5,4.0,2.1,33.9,"MVP-3,CPOY-6,AS,NBA1"
2,Giannis Antetokounmpo,29.0,MIL,PF,73.0,73.0,35.2,11.5,18.8,0.611,...,2.7,8.8,11.5,6.5,1.2,1.1,3.4,2.9,30.4,"MVP-4,DPOY-9,CPOY-12,AS,NBA1"
3,Shai Gilgeous-Alexander,25.0,OKC,PG,75.0,75.0,34.0,10.6,19.8,0.535,...,0.9,4.7,5.5,6.2,2.0,0.9,2.2,2.5,30.1,"MVP-2,DPOY-7,CPOY-3,AS,NBA1"
4,Jalen Brunson,27.0,NYK,PG,77.0,77.0,35.4,10.3,21.4,0.479,...,0.6,3.1,3.6,6.7,0.9,0.2,2.4,1.9,28.7,"MVP-5,CPOY-5,AS,NBA2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,Ron Harper Jr.,23.0,TOR,PF,1.0,0.0,4.0,0.0,0.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,NaN
732,Justin Jackson,28.0,MIN,SF,2.0,0.0,0.5,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
733,Dmytro Skapintsev,25.0,NYK,C,2.0,0.0,1.0,0.0,0.5,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
734,Javonte Smart,24.0,PHI,PG,1.0,0.0,1.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


Ker imamo veliko kategorij podatkov, jih bomo nekaj izbrisali, da bo tabela preglednejša. Izbrisali bomo stolpce, ki označujejo podatke za odigrane tekme v prvi postavi, zadeti meti za dve točki, število poskusov metov za dve točki, procentna uspešnost zadetih metov za dve točki, eFG%, število prostih metov, število izvedenih prostih metov, skoki v napadu, skoki v obrambi in nagrade. 
Opomba: večina podatkov je beležena kot povprečje na tekmo.

In [4]:
redni_del = redni_del.drop(columns=["GS", "2P", "2PA", "2P%", "eFG%", "FT", "FTA", "ORB", "DRB", "Awards"])
redni_del

,Player,Age,Team,Pos,G,MP,FG,FGA,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
0,Joel Embiid,29.0,PHI,C,39.0,33.6,11.5,21.8,0.529,1.4,3.6,0.388,0.883,11.0,5.6,1.2,1.7,3.8,2.9,34.7
1,Luka Dončić,24.0,DAL,PG,70.0,37.5,11.5,23.6,0.487,4.1,10.6,0.382,0.786,9.2,9.8,1.4,0.5,4.0,2.1,33.9
2,Giannis Antetokounmpo,29.0,MIL,PF,73.0,35.2,11.5,18.8,0.611,0.5,1.7,0.274,0.657,11.5,6.5,1.2,1.1,3.4,2.9,30.4
3,Shai Gilgeous-Alexander,25.0,OKC,PG,75.0,34.0,10.6,19.8,0.535,1.3,3.6,0.353,0.874,5.5,6.2,2.0,0.9,2.2,2.5,30.1
4,Jalen Brunson,27.0,NYK,PG,77.0,35.4,10.3,21.4,0.479,2.7,6.8,0.401,0.847,3.6,6.7,0.9,0.2,2.4,1.9,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,Ron Harper Jr.,23.0,TOR,PF,1.0,4.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,2.0,0.0
732,Justin Jackson,28.0,MIN,SF,2.0,0.5,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
733,Dmytro Skapintsev,25.0,NYK,C,2.0,1.0,0.0,0.5,0.000,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
734,Javonte Smart,24.0,PHI,PG,1.0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# izbrišimo zadnjo vrstico, ki označuje League average
redni_del = redni_del.drop(redni_del.index[-1])

V tabeli so nekateri igralci zapisani večkrat, ker so v sezoni zaigrali za dve ali pa celo tri ekipe. Poiščimo te igralce in preuredimo tabelo tako,
da bodo vsi igralci zapisani samo enkrat.

In [6]:
twoteams_players = redni_del[redni_del["Team"] == "2TM"]
twoteams_players

,Player,Age,Team,Pos,G,MP,FG,FGA,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
37,Pascal Siakam,29.0,2TM,PF,80.0,33.2,8.5,15.9,0.536,1.1,3.1,0.346,0.732,7.1,4.3,0.8,0.3,1.8,2.4,21.7
48,RJ Barrett,23.0,2TM,SG,58.0,31.7,7.5,15.2,0.495,1.6,4.3,0.360,0.715,5.4,3.3,0.5,0.4,2.2,2.2,20.2
55,Terry Rozier,29.0,2TM,SG,61.0,33.4,7.2,16.4,0.443,2.4,6.7,0.363,0.869,4.0,5.6,1.0,0.3,1.7,1.7,19.8
79,Immanuel Quickley,24.0,2TM,SG,68.0,29.2,5.7,13.1,0.434,2.5,6.4,0.395,0.853,3.8,4.9,0.7,0.1,1.5,1.8,17.0
100,Bojan Bogdanović,34.0,2TM,PF,57.0,25.9,5.4,11.9,0.454,2.3,5.9,0.398,0.785,2.7,1.7,0.5,0.1,1.8,1.7,15.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Ish Wainright,29.0,2TM,SF,11.0,5.6,0.6,2.2,0.292,0.6,1.8,0.350,1.000,1.3,0.1,0.4,0.1,0.2,0.7,2.1
650,Mamadi Diakite,27.0,2TM,PF,6.0,4.0,0.7,1.0,0.667,0.0,0.2,0.000,0.667,0.7,0.3,0.2,0.2,0.2,0.3,2.0
666,Taj Gibson,38.0,2TM,PF,20.0,10.2,0.8,1.9,0.405,0.1,0.5,0.200,1.000,1.9,0.6,0.2,0.4,0.3,1.6,1.7
669,P.J. Tucker,38.0,2TM,PF,31.0,15.7,0.6,1.6,0.360,0.4,1.1,0.371,1.000,2.7,0.5,0.5,0.2,0.3,1.7,1.7


In [7]:
threeteams_players = redni_del[redni_del["Team"] == "3TM"]
threeteams_players

,Player,Age,Team,Pos,G,MP,FG,FGA,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
386,Danilo Gallinari,35.0,3TM,SF,49.0,12.9,1.7,3.9,0.437,0.6,1.9,0.323,0.857,2.2,1.1,0.3,0.1,0.5,1.0,5.7
396,Malachi Flynn,25.0,3TM,PG,69.0,12.7,2.0,4.7,0.418,0.8,2.4,0.331,0.732,1.7,1.9,0.6,0.1,0.9,1.0,5.5
458,Kenneth Lofton Jr.,21.0,3TM,PF,21.0,9.5,1.7,3.7,0.462,0.3,1.1,0.292,0.654,1.8,1.6,0.3,0.2,0.7,1.1,4.6
465,Shake Milton,27.0,3TM,SG,48.0,12.1,1.6,4.1,0.405,0.4,1.3,0.281,0.816,1.6,1.3,0.4,0.1,0.6,1.1,4.5
579,Kira Lewis Jr.,22.0,3TM,PG,28.0,9.5,1.2,3.3,0.370,0.1,1.2,0.121,0.850,0.9,1.3,0.3,0.1,0.5,0.5,3.2
592,Mike Muscala,32.0,3TM,C,53.0,11.3,1.1,2.9,0.359,0.5,1.8,0.292,0.680,2.4,0.7,0.2,0.2,0.4,1.0,3.0
615,Dylan Windler,27.0,3TM,SF,17.0,6.4,0.9,1.8,0.500,0.8,1.6,0.481,NaN,0.9,0.6,0.1,0.0,0.1,0.4,2.5


In [10]:
# koliko igralcev je sezono igralo samo za eno ekipo
singleteam_players = redni_del.drop_duplicates(subset=['Player'], keep=False)
stevilo_singleteam_players = len(singleteam_players)
stevilo_singleteam_players

494

In [12]:
# prečiščena tabela, vsak igralec zapisan samo enkrat
redni_del = pd.concat([singleteam_players, twoteams_players, threeteams_players])
redni_del

,Player,Age,Team,Pos,G,MP,FG,FGA,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
0,Joel Embiid,29.0,PHI,C,39.0,33.6,11.5,21.8,0.529,1.4,3.6,0.388,0.883,11.0,5.6,1.2,1.7,3.8,2.9,34.7
1,Luka Dončić,24.0,DAL,PG,70.0,37.5,11.5,23.6,0.487,4.1,10.6,0.382,0.786,9.2,9.8,1.4,0.5,4.0,2.1,33.9
2,Giannis Antetokounmpo,29.0,MIL,PF,73.0,35.2,11.5,18.8,0.611,0.5,1.7,0.274,0.657,11.5,6.5,1.2,1.1,3.4,2.9,30.4
3,Shai Gilgeous-Alexander,25.0,OKC,PG,75.0,34.0,10.6,19.8,0.535,1.3,3.6,0.353,0.874,5.5,6.2,2.0,0.9,2.2,2.5,30.1
4,Jalen Brunson,27.0,NYK,PG,77.0,35.4,10.3,21.4,0.479,2.7,6.8,0.401,0.847,3.6,6.7,0.9,0.2,2.4,1.9,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Kenneth Lofton Jr.,21.0,3TM,PF,21.0,9.5,1.7,3.7,0.462,0.3,1.1,0.292,0.654,1.8,1.6,0.3,0.2,0.7,1.1,4.6
465,Shake Milton,27.0,3TM,SG,48.0,12.1,1.6,4.1,0.405,0.4,1.3,0.281,0.816,1.6,1.3,0.4,0.1,0.6,1.1,4.5
579,Kira Lewis Jr.,22.0,3TM,PG,28.0,9.5,1.2,3.3,0.370,0.1,1.2,0.121,0.850,0.9,1.3,0.3,0.1,0.5,0.5,3.2
592,Mike Muscala,32.0,3TM,C,53.0,11.3,1.1,2.9,0.359,0.5,1.8,0.292,0.680,2.4,0.7,0.2,0.2,0.4,1.0,3.0


V tabeli najdemo igralce, ki imajo med svojimi podatki dokaj veliko ničelnih kategorij ali pa so odigrali majhno število tekem oziroma so dobili nizko stopnjo minutaže.
Za nadaljno analizo izločimo igralce, ki so odigrali manj kot 5 tekem v sezoni in tiste, ki so bili na igrišču manj kot 4 minute na tekmo.

In [13]:
redni_del = redni_del[(redni_del['G'] >= 5) & (redni_del['MP'] >= 4)]
redni_del

,Player,Age,Team,Pos,G,MP,FG,FGA,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS
0,Joel Embiid,29.0,PHI,C,39.0,33.6,11.5,21.8,0.529,1.4,3.6,0.388,0.883,11.0,5.6,1.2,1.7,3.8,2.9,34.7
1,Luka Dončić,24.0,DAL,PG,70.0,37.5,11.5,23.6,0.487,4.1,10.6,0.382,0.786,9.2,9.8,1.4,0.5,4.0,2.1,33.9
2,Giannis Antetokounmpo,29.0,MIL,PF,73.0,35.2,11.5,18.8,0.611,0.5,1.7,0.274,0.657,11.5,6.5,1.2,1.1,3.4,2.9,30.4
3,Shai Gilgeous-Alexander,25.0,OKC,PG,75.0,34.0,10.6,19.8,0.535,1.3,3.6,0.353,0.874,5.5,6.2,2.0,0.9,2.2,2.5,30.1
4,Jalen Brunson,27.0,NYK,PG,77.0,35.4,10.3,21.4,0.479,2.7,6.8,0.401,0.847,3.6,6.7,0.9,0.2,2.4,1.9,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Kenneth Lofton Jr.,21.0,3TM,PF,21.0,9.5,1.7,3.7,0.462,0.3,1.1,0.292,0.654,1.8,1.6,0.3,0.2,0.7,1.1,4.6
465,Shake Milton,27.0,3TM,SG,48.0,12.1,1.6,4.1,0.405,0.4,1.3,0.281,0.816,1.6,1.3,0.4,0.1,0.6,1.1,4.5
579,Kira Lewis Jr.,22.0,3TM,PG,28.0,9.5,1.2,3.3,0.370,0.1,1.2,0.121,0.850,0.9,1.3,0.3,0.1,0.5,0.5,3.2
592,Mike Muscala,32.0,3TM,C,53.0,11.3,1.1,2.9,0.359,0.5,1.8,0.292,0.680,2.4,0.7,0.2,0.2,0.4,1.0,3.0
